In [23]:
import os
import pandas as pd
# data = pd.read_csv('normal_state_2020-0907.csv')
data = pd.read_csv('normal.csv')
# data.sort_values(by=['segment','time'], inplace=True)
data.head(10)

,time,icao24,lat,lon,baroaltitude
0,-28800,0,0.712793,0.113518,0.108725
1,-28800,1,0.708481,0.113015,0.117587
2,-28800,2,0.712011,0.114348,0.327880
3,-28790,0,0.712936,0.113526,0.113838
4,-28790,1,0.708366,0.113003,0.116564
5,-28790,2,0.712173,0.114337,0.331288
6,-28790,3,0.785423,0.523083,0.249148
7,-28790,4,0.680608,0.605421,0.522836
8,-28790,5,0.697623,0.557910,0.536810
9,-28790,6,0.786225,0.272312,0.156442


# 优化数据集

In [24]:
id_list = data['icao24'].unique()
abandon_id = [5, 10, 26, 131, 149, 156, 179, 190, 191,192, 216, 225, 262, 268, 329, 11, 53, 67, 83, 34, 62, 146, 172, 175, 185, 245, 228, 229]
icao24_list = [id for id in id_list if id not in abandon_id]
print(len(icao24_list))
data = data[data['icao24'].isin(icao24_list)]
data.info()

313
<class 'pandas.core.frame.DataFrame'>
Int64Index: 70900 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time          70900 non-null  int64  
 1   icao24        70900 non-null  int64  
 2   lat           70900 non-null  float64
 3   lon           70900 non-null  float64
 4   baroaltitude  70900 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 3.2 MB


## 可视化


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def show_3D(obs_traj):
    fig = plt.figure()
    ax = Axes3D(fig)
    line_width = 0.5
    ax.plot(obs_traj[:, 2], obs_traj[:, 3], obs_traj[:, 4], color='r', label='traj',linewidth=line_width)
    ax.legend()
    plt.show()

In [26]:
# id_list = data['icao24'].unique()
# for id in id_list:
#     plt_data = data[data['icao24'] ==id]
#     print(id)
#     show_3D(np.array(plt_data))

# 验证

In [27]:
grouped=data.groupby('icao24')
for name,group in grouped:
    if (group['time'].iloc[0]) + (len(group['time']) - 1) * 10 != group['time'].iloc[-1]:
        print (group['icao24'].unique())
        print (group['time'].iloc[-1])
        print (len(group['time']))

# 数据集划分

In [28]:
## 将训练集、验证集、测试集按照7:2:1划分

train_row = int(.7*len(data))
val_row = int(.9*len(data))

train = data.iloc[:train_row]
val = data.iloc[train_row:val_row]
test = data.iloc[val_row:]
# 统计航迹点
train['icao24'].value_counts()
val['icao24'].value_counts()

319    99
4      99
19     99
35     99
43     99
       ..
7       5
318     4
275     3
292     3
271     2
Name: icao24, Length: 177, dtype: int64

In [29]:
# 过滤掉轨迹长度小于30的点
train = train.groupby(by=[train['icao24']]).filter(lambda x: len(x) >= 30)
val = val.groupby(by=[val['icao24']]).filter(lambda x: len(x) >= 30)
test = test.groupby(by=[test['icao24']]).filter(lambda x: len(x) >= 30)

In [30]:
train['icao24'].value_counts()

0      193
2      193
107    192
43     192
54     192
      ... 
209     40
283     38
325     38
14      36
49      31
Name: icao24, Length: 313, dtype: int64

In [31]:
val['icao24'].value_counts()


319    99
295    99
3      99
19     99
35     99
       ..
240    37
45     37
124    36
152    34
291    32
Name: icao24, Length: 150, dtype: int64

In [32]:
test['icao24'].value_counts()

323    69
304    69
319    69
114    69
130    69
       ..
249    43
106    42
242    36
232    34
119    31
Name: icao24, Length: 106, dtype: int64

In [33]:
dataset_path = 'states20210830_small'
train_path = os.path.join(dataset_path,"train")
val_path = os.path.join(dataset_path,"val")
test_path = os.path.join(dataset_path,"test")

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(val_path):
    os.makedirs(val_path)

if not os.path.exists(test_path):
    os.makedirs(test_path)

train_file = os.path.join(train_path, 'train.txt')
val_file = os.path.join(val_path, 'val.txt')
test_file = os.path.join(test_path, 'test.txt')
# 保存文件
train.to_csv(train_file, index=False, sep='\t', header=0)
val.to_csv(val_file, index=False, sep='\t',header=0)
test.to_csv(test_file, index=False, sep='\t',header=0)

def process_txt(filename):
    with open(filename, "r+") as csv_file:
        content = csv_file.read()
    with open(filename, "w+") as csv_file:
        csv_file.write(content.replace('"', ''))
process_txt("{}/train/train.txt".format(dataset_path))
process_txt("{}/test/test.txt".format(dataset_path))
process_txt("{}/val/val.txt".format(dataset_path))#%%

将数据只保留时间、icao24、经度、纬度、高度

In [34]:
# import pandas as pd

# data = pd.read_csv('normal_climb_H8D_train.csv', usecols=['time', 'icao24', 'lat', 'lon', 'baroaltitude'])
# data.to_csv("normal_climb_H8D_train_3d.csv", index=False)

